Bastián Marinkovic

# Set Up

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label
from scipy.ndimage import binary_fill_holes

# Decargar las imágenes

In [2]:
!wget https://www.dropbox.com/s/qix65nxcevypw0a/T01_imagenes.zip?dl=0
!unzip -qq T01_imagenes.zip?dl=0
# !ls T01_imagenes

--2023-04-03 05:14:12--  https://www.dropbox.com/s/qix65nxcevypw0a/T01_imagenes.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/qix65nxcevypw0a/T01_imagenes.zip [following]
--2023-04-03 05:14:13--  https://www.dropbox.com/s/raw/qix65nxcevypw0a/T01_imagenes.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf1b7fe32dd61fc12d965cb2207.dl.dropboxusercontent.com/cd/0/inline/B5fuxO3FpncWj-DTZBQ38SFV_E3Sj12TCVdv3U0L50YNTGaU9vIQUfO-p0SajEpT3cjtQKjMCeeVrK_Ai6KeOsapSpVdruGYTRCeJ0jP6sACwiZDyk_r4-SuFsDE-VlUt4f-ednPtIXgwrpjGF7YzF6PcLXMcu9I2--V0nRl3RBNug/file# [following]
--2023-04-03 05:14:13--  https://ucf1b7fe32dd61fc12d965cb2207.dl.dropboxusercontent.com/cd/0/inline/B5fuxO3FpncWj-DTZBQ38SFV_E3Sj12TCVdv3U0L50YNTGaU9vIQUfO-p0SajEp

# Funciones para cargar las imágenes

In [3]:
# Funciones necesarias para que se pueda cargar una imagen individual

def num2fixstr(x,d):
  # example num2fixstr(2,5) returns '00002'
  # example num2fixstr(19,3) returns '019'
  st = '%0*d' % (d,x)
  return st

def ImageLoad(prefix,num_img,echo='off'):
  # img = ImageLoad('example/char_',1,3)   loads image 'example/char_01_003.png'
  # img = ImageLoad('example/char_',2,15)  loads image 'example/char_02_015.png'
  st   = prefix + num2fixstr(num_img, 2) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = plt.imread(st)
  return img

# Identificación de cada número presente en la imagen

## Funciones auxiliares para reconocer los números

In [4]:
def Determine_number(img, x, y, w, h):

  """
  Función encargada de determinar el número presente en la imangen, recibe
  la imagen completa y las coordenadas del número a analizar (la bounding box
  que lo comprende). 
     
  Retorna el número determinado en la bounding box.
  """

  # bounding box
  bbox = img[y:y+h, x:x+w]

  # umbralización
  bbox[bbox >  0.25 * 255] = 255
  bbox[bbox <= 0.25 * 255] = 0
  
  # los siguientes valores son para chequear cuanta area de la bounding box
  # cubre un numero y lo mismo a nivel de linea
  percentageNumberOne = 0.97
  percentageLine = 0.9
  
  # número es 1 si el area pintada representa mas que percentageNumberOne del area total
  if checkAreaPercentage(bbox, w, h, percentageNumberOne):
    return 1
  
  # si el número tiene agujeros
  if haveHoles(bbox): # 0, 4, 8, 9 
    # como la imagen tiene orificios se tiene que determinar si es que están 
    # presentes solamantenete en la mitad inforior de la imagen, en la superior
    # o en ambas mitades
    
    haveUpperHoles = haveHoles(bbox[:int(h / 2), :]) # agujeros mitad superior
    haveLowerHoles = haveHoles(bbox[int(h / 2):, :]) # agujeros mitad inferior

    if haveUpperHoles and haveLowerHoles:
      return 8
    elif haveUpperHoles and not(haveLowerHoles):
      return 9
    elif not(haveUpperHoles) and haveLowerHoles:
      return 6
    
    """
    si se llega a este punto es porque es un 4 o un 0, esto porque ambos numeros
    presentan agujeros, pero se encuentran al mismo tiempo en la mitad
    superior e inferior. 

    se comprueba si es un 4 o no, para esto se ve si el area percentage de dos
    lineas que cruzan el numero, si ambas cumplen es 4, sino es 0
    """
    
    axis = 0
    axis2 = 1
    start = 0
    end = 1

    # lineas que se realizaran la operación, se ve sobre una linea horizontal y
    # una vertical (orientación definida por axis)
    lineVertical, hV, wV = createLine(bbox, axis, start, end)
    lineHorizontal, hH, wH = createLine(bbox, axis2, start, end)

    percentageVert = checkAreaPercentage(lineVertical, wV, h, percentageLine)
    percentageHori = checkAreaPercentage(lineHorizontal, w, hH, percentageLine)

    if percentageVert and percentageHori:
      return 4
    
    return 0
    

  else: # 2, 3, 5, 7
    
    # si el area cubierta en la base del numero es mayor a percentageLine es un 2
    if checkAreaPercentage(bbox[h-1, :], w, 1, percentageLine):
      return 2
    
    # si el area cubierta en la parte superior del numero es mayor a percentageLine es un 7
    if checkAreaPercentage(bbox[0, :], w, 1, percentageLine):
      return 7
    
    # para determinar si es un 3 o un 5 se ve la diferencia de areas pintadas
    # con blanco de la parte izquierda con la parte derecha de la imagen, si la
    # diferencia porcentual entre ambas es mayor a un 20% es un 3, sino es 5

    leftArea = np.sum(bbox[:, :int(w / 2)]) / 255
    rightArea = np.sum(bbox[:, int(w / 2):]) / 255

    diff = abs(leftArea - rightArea)

    percentageDiff = 100 * diff / max(leftArea, rightArea)

    if percentageDiff < 20:
      return 5
    
    return 3

In [5]:
def checkAreaPercentage(bbox, w, h, percentage):
  """
  funcion que recibe la bound box, su ancho y alto comprueba si el área 
  pintada es mayor a un percentage del area total, si cumple retorna true, 
  sino retorna false
  """
  if np.sum(bbox) >= ( 255 * w * h * percentage ):
    return True
  return False

In [6]:
def haveHoles(bbox):
  """
  Esta función recibe el bounding box
  
  Retorna un bool, es true tiene orificio, false en otro caso
  """
  area = int(np.sum(bbox / 255))
  filledArea = FilledArea(bbox)
  if area == filledArea:
    return False
  return True


def FilledArea(img):
  """
  Codigo basado de colab visto en clase, separación jq, retorna el área de la 
  imagen luego de que se aplique la operación binary_fill_holes
  """
  R = binary_fill_holes(img).astype(int) 
  NumArea = np.sum(R)                    
  return NumArea


In [7]:
def findInterestCoordinates(line):
  """
  Función que retorna las coordenadas del primer pixel pintado en la imagen
  """
  for i in range(line.shape[0]):
    for j in range(line.shape[1]):
      if line[i, j] == 255:
        return [[i, j]]

def createLine(bbox, axis, start, end):
  """
  funcion que toma como argumento la bounding box, y crea una linea en el axis
  indicado con los puntos de inicio y fin, sobre esta linea se realiza una busqueda
  de pixeles pintados, y en base a las coordenadas del primer pixel pintado se
  crea una segunda linea desde dicho primer pixel hasta el tamaño de la imagen 
  en el sentido correspondiente. 

  La función retorna una linea, su alto y ancho.
  """
  
  if axis == 0:
    line = bbox[start:end, :] # extremo del numero sobre el que se hara la busqueda

  else:
    line = bbox[:, start:end] # extremo del numero sobre el que se hara la busqueda

  idLine = findInterestCoordinates(line)

  if axis == 0: # se crea una linea vertical
    interestLine = bbox[idLine[0][0]:, idLine[0][1]]    # linea de interes
    interestLine = interestLine.reshape(bbox.shape[0], 1)
  
  else: # se crea una linea horizontal
    interestLine = bbox[idLine[0][0], idLine[0][1]:]    # linea de interes 
    interestLine = interestLine.reshape(1, bbox.shape[1]) 

  h, w = interestLine.shape
  return interestLine, h, w

In [8]:
def findBoundingBox(img, color):
  """
  Función que recibe imagen luego de que se pasase por label y el color del
  numero sobre el que se busca la bounding box
  retorna las coordenadas del inicio de la bbox, su ancho y alto
  """

  imin = img.shape[0]
  imax = 0
  jmin = img.shape[1]
  jmax = 0
  for i in range(img.shape[0]):
    if np.any(img[i, :] != 0):     # optimización sobre la busqueda, alternativamente se puede hacer if np.sum(img[i, :]) != 0:
      for j in range(img.shape[1]):
        if np.any(img[:, j] != 0): # optimización sobre la busqueda, alternativamente se puede hacer if np.sum(img[:, j]) != 0:
          if img[i,j] == color:
            if i<imin:
              imin = i
            if i>imax:
              imax = i
            if j<jmin:
              jmin = j
            if j>jmax:
              jmax = j
  return jmin, imin, jmax - jmin + 1, imax - imin + 1

## Definición de reconocedor

In [9]:
def Reconocedor(img):
  """
  Función que recibe imagen cargada por ImageLoad
  retorna lista de numeros encontrados
  """

  # Los valores de la imagen se dejan entre [0, 255]
  img = img * 255
  
  # Se pasa a escala de grises y se juntan los 3 canales de colores en uno solamente
  gray = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2GRAY)

  # se filtran los pixeles menores a 50
  R = gray > 50
  thresh = R.astype(int)
  # se genera labels de los numeros presentes en la imagen con el fin de identificar
  # cada numero individualmente
  L = label(R)
  bounding_boxes = []
  for i in range(1, np.max(L) + 1):
    x, y, w, h = findBoundingBox(L, i)
    bounding_boxes.append((x, y, w, h))

  # se ordenan los números respecto a el orden de aparicion en la imagen
  bounding_boxes = sorted(bounding_boxes, key=lambda x: x[0])

  numbersDetermine = []
  
  for bbox in bounding_boxes: # se determina el numero presente en cada bbox
      x, y, w, h = bbox
      numbersDetermine.append(Determine_number(gray, x, y, w, h))
  
  return numbersDetermine

In [10]:
# etiquetas de imagenes
groundTruth = [
    [4, 6, 1, 1, 2, 9],
    [9, 5, 2, 9],
    [4, 5, 1, 9, 0],
    [6, 2, 8, 0, 9, 3],
    [2, 3, 6, 9, 1],
    [4, 2, 8, 7, 2, 3, 9],
    [8, 6, 3, 2, 9],
    [3, 5, 1, 9, 0, 8],
    [7, 2, 8, 7, 0, 9, 3],
    [2, 8, 2, 5, 2, 9],
    [2, 9, 1, 1, 2],
    [2, 9, 9, 5],
    [5, 1, 9, 0, 4, 5],
    [8, 0, 8, 0, 9, 3, 6, 2, 8],
    [2, 3, 6, 9, 1, 6, 2, 3],
    [8, 7, 2, 3, 9],
    [6, 8, 6, 3, 2, 9, 8],
    [5, 3, 5, 1, 9, 0, 8, 1, 9],
    [2, 8, 7, 0, 9],
    [2, 8, 2, 5, 2, 9, 2, 5]
]

## Ejecución de roconocedor

In [11]:
results = []
for i in range(1,21):
  img    = ImageLoad('prueba_',i,echo='off')
  results.append(Reconocedor(img))

# Matriz de confusión

In [12]:
matrix = [[0 for _ in range(10)] for _ in range(10)]

In [13]:
total = 0
correct = 0
for image in range(len(results)):
  quantityOfNumbers = len(groundTruth[image])
  total += quantityOfNumbers

  for number in range(quantityOfNumbers):
    matrix[groundTruth[image][number]][results[image][number]] += 1
    if groundTruth[image][number] == results[image][number]:
      correct += 1

In [14]:
matrix

[[9, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 11, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 24, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 12, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 9, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 15, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 23]]

In [15]:
acc = correct / total * 100
acc

100.0